# Importing necessary libraries

In [1]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from git import Repo

# Cloning from repository

In [2]:
repo_url = "https://github.com/PhonePe/pulse.git"
clone_path = r"C:\ProgramFiles\Projects\Phonepe_pulse"

if not os.path.exists(clone_path):
    os.makedirs(clone_path)

repo_path = os.path.join(clone_path, os.path.basename(repo_url).removesuffix('.git').title())

Repo.clone_from(repo_url, repo_path)

directory = os.path.join(repo_path, 'data')
print(directory)

C:\ProgramFiles\Projects\Phonepe_pulse\Pulse\data


# Renaming sub-directories and Extracting necessary paths

In [3]:
# Function to rename messy state names in a proper format

def rename(directory):
    for root, dirs, files in os.walk(directory):
        if 'state' in dirs:
            state_dir = os.path.join(root, 'state')
            for state_folder in os.listdir(state_dir):
                # rename the state folder
                old_path = os.path.join(state_dir, state_folder)
                new_path = os.path.join(state_dir, state_folder.title().replace('-', ' ').replace('&', 'and'))
                os.rename(old_path, new_path)
    print("Renamed all sub-directories successfully")
                
# Function to extract all paths that has sub-directory in the name of 'state'

def extract_paths(directory):
    path_list = []
    for root, dirs, files in os.walk(directory):
        if os.path.basename(root) == 'state':
            path_list.append(root.replace('\\', '/'))
    return path_list

In [4]:
rename(directory)

Renamed all sub-directories successfully


In [5]:
state_directories = extract_paths(directory)
state_directories

['C:/ProgramFiles/Projects/Phonepe_pulse/Pulse/data/aggregated/transaction/country/india/state',
 'C:/ProgramFiles/Projects/Phonepe_pulse/Pulse/data/aggregated/user/country/india/state',
 'C:/ProgramFiles/Projects/Phonepe_pulse/Pulse/data/map/transaction/hover/country/india/state',
 'C:/ProgramFiles/Projects/Phonepe_pulse/Pulse/data/map/user/hover/country/india/state',
 'C:/ProgramFiles/Projects/Phonepe_pulse/Pulse/data/top/transaction/country/india/state',
 'C:/ProgramFiles/Projects/Phonepe_pulse/Pulse/data/top/user/country/india/state']

# Creating dataframes from cloned json files

1. Aggregate Transaction

In [6]:
state_path = state_directories[0]
state_list = os.listdir(state_path)
agg_trans_dict = {
                  'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [],
                  'Transaction_count': [], 'Transaction_amount': []
                  }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for transaction_data in df['data']['transactionData']:
                    
                    type = transaction_data['name']
                    count = transaction_data['paymentInstruments'][0]['count']
                    amount = transaction_data['paymentInstruments'][0]['amount']
                    
                    # Appending to agg_trans_dict
                    
                    agg_trans_dict['State'].append(state)
                    agg_trans_dict['Year'].append(year)
                    agg_trans_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    agg_trans_dict['Transaction_type'].append(type)
                    agg_trans_dict['Transaction_count'].append(count)
                    agg_trans_dict['Transaction_amount'].append(amount)
            except:
                pass
            
agg_trans_df = pd.DataFrame(agg_trans_dict)

2. Aggregate User

In [7]:
state_path = state_directories[1]
state_list = os.listdir(state_path)
agg_user_dict = {
                 'State': [], 'Year': [], 'Quarter': [], 'Brand': [],
                 'Transaction_count': [], 'Percentage': []
                 }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for user_data in df['data']['usersByDevice']:

                    brand = user_data['brand']
                    count = user_data['count']
                    percent = user_data['percentage']
                    
                    # Appending to agg_user_dict
                    
                    agg_user_dict['State'].append(state)
                    agg_user_dict['Year'].append(year)
                    agg_user_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    agg_user_dict['Brand'].append(brand)
                    agg_user_dict['Transaction_count'].append(count)
                    agg_user_dict['Percentage'].append(percent)
            except:
                pass

agg_user_df = pd.DataFrame(agg_user_dict)

3. Map Transaction

In [8]:
state_path = state_directories[2]
state_list = os.listdir(state_path)
map_trans_dict = {
                    'State': [], 'Year': [], 'Quarter': [], 'District': [],
                    'Transaction_count': [], 'Transaction_amount': []
                    }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for transaction_data in df['data']['hoverDataList']:
                   
                    district = transaction_data['name']
                    count = transaction_data['metric'][0]['count']
                    amount = transaction_data['metric'][0]['amount']
                    
                    # Appending to map_trans_dict
                    
                    map_trans_dict['State'].append(state)
                    map_trans_dict['Year'].append(year)
                    map_trans_dict['Quarter'].append(int(quarter.removesuffix('.json'))) 
                    map_trans_dict['District'].append(district.removesuffix(' district').title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    map_trans_dict['Transaction_count'].append(count)
                    map_trans_dict['Transaction_amount'].append(amount)
            except:
                pass

map_trans_df = pd.DataFrame(map_trans_dict)

4. Map User

In [9]:
state_path = state_directories[3]
state_list = os.listdir(state_path)
map_user_dict = {
                 'State': [], 'Year': [], 'Quarter': [], 'District': [],
                 'Registered_users': [], 'App_opens': []
                 }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for district, user_data in df['data']['hoverData'].items():
                    
                    reg_user_count = user_data['registeredUsers']
                    app_open_count = user_data['appOpens']
                    
                    # Appending to map_user_dict
                    
                    map_user_dict['State'].append(state)
                    map_user_dict['Year'].append(year)
                    map_user_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    map_user_dict['District'].append(district.removesuffix(' district').title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    map_user_dict['Registered_users'].append(reg_user_count)
                    map_user_dict['App_opens'].append(app_open_count)
            except:
                pass

map_user_df = pd.DataFrame(map_user_dict)

5. Top Transaction District-wise

In [10]:
state_path = state_directories[4]
state_list = os.listdir(state_path)
top_trans_dist_dict = {
                        'State': [], 'Year': [], 'Quarter': [], 'District': [],
                        'Transaction_count': [], 'Transaction_amount': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for district_data in df['data']['districts']:
                    
                    name = district_data['entityName']
                    count = district_data['metric']['count']
                    amount = district_data['metric']['amount']
                    
                    # Appending to top_trans_dist_dict
                    
                    top_trans_dist_dict['State'].append(state)
                    top_trans_dist_dict['Year'].append(year)
                    top_trans_dist_dict['Quarter'].append(int(quarter.removesuffix('.json')))                    
                    top_trans_dist_dict['District'].append(name.title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    top_trans_dist_dict['Transaction_count'].append(count)
                    top_trans_dist_dict['Transaction_amount'].append(amount)
            except:
                pass

top_trans_dist_df = pd.DataFrame(top_trans_dist_dict)

6. Top Transaction Pincode-wise

In [11]:
state_path = state_directories[4]
state_list = os.listdir(state_path)
top_trans_pin_dict = {
                        'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
                        'Transaction_count': [], 'Transaction_amount': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for regional_data in df['data']['pincodes']:
                    
                    name = regional_data['entityName']
                    count = regional_data['metric']['count']
                    amount = regional_data['metric']['amount']
                    
                    # Appending to top_trans_pin_dict
                    
                    top_trans_pin_dict['State'].append(state)
                    top_trans_pin_dict['Year'].append(year)
                    top_trans_pin_dict['Quarter'].append(int(quarter.removesuffix('.json')))                    
                    top_trans_pin_dict['Pincode'].append(name)
                    top_trans_pin_dict['Transaction_count'].append(count)
                    top_trans_pin_dict['Transaction_amount'].append(amount)
            except:
                pass

top_trans_pin_df = pd.DataFrame(top_trans_pin_dict)

7. Top User District-wise

In [12]:
state_path = state_directories[5]
state_list = os.listdir(state_path)
top_user_dist_dict = {
                        'State': [], 'Year': [], 'Quarter': [],
                        'District': [], 'Registered_users': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for district_data in df['data']['districts']:
                    
                    name = district_data['name']
                    count = district_data['registeredUsers']
                    
                    # Appending to top_user_dist_dict
                    
                    top_user_dist_dict['State'].append(state)
                    top_user_dist_dict['Year'].append(year)
                    top_user_dist_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    top_user_dist_dict['District'].append(name.title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    top_user_dist_dict['Registered_users'].append(count)
            except:
                pass

top_user_dist_df = pd.DataFrame(top_user_dist_dict)

8. Top User Pincode-wise

In [13]:
state_path = state_directories[5]
state_list = os.listdir(state_path)
top_user_pin_dict = {
                        'State': [], 'Year': [], 'Quarter': [],
                        'Pincode': [], 'Registered_users': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for regional_data in df['data']['pincodes']:
                    
                    name = regional_data['name']
                    count = regional_data['registeredUsers']
                    
                    # Appending to top_user_pin_dict
                    
                    top_user_pin_dict['State'].append(state)
                    top_user_pin_dict['Year'].append(year)
                    top_user_pin_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    top_user_pin_dict['Pincode'].append(name)
                    top_user_pin_dict['Registered_users'].append(count)
            except:
                pass

top_user_pin_df = pd.DataFrame(top_user_pin_dict)

# List of dataframes created so far

In [14]:
df_list = [df for df in globals() if isinstance(globals()[df], pd.core.frame.DataFrame) and df.endswith('_df')]

df_list

['agg_trans_df',
 'agg_user_df',
 'map_trans_df',
 'map_user_df',
 'top_trans_dist_df',
 'top_trans_pin_df',
 'top_user_dist_df',
 'top_user_pin_df']

# Renaming Delhi districts to manage inconsistency

In [15]:
# I noticed few district name is mismatched between dfs loaded from pulse and lat_long_df, doing this.

def add_suffix_to_districts(df):
    if 'District' in df.columns and 'State' in df.columns:
        delhi_df = df[df['State'] == 'Delhi']
        
        districts_to_suffix = [d for d in delhi_df['District'].unique() if d != 'Shahdara']
        
        df.loc[(df['State'] == 'Delhi') & (df['District'].isin(districts_to_suffix)), 'District'] = df.loc[(df['State'] == 'Delhi') & (df['District'].isin(districts_to_suffix)), 'District'].apply(lambda x: x + ' Delhi' if 'Delhi' not in x else x)

    return df

for df_name in df_list:
    df = globals()[df_name]
    add_suffix_to_districts(df)

# Adding Latitude and Longitude columns

In [16]:
lat_long_df = pd.read_csv(r"ExtData\dist_lat_long.csv")

for df_name in df_list:
    df = globals()[df_name]
    if 'District' in df.columns:
        df = pd.merge(df, lat_long_df, on=['State', 'District'], how='left')
        globals()[df_name] = df

# Adding Region column to all dataframes

In [17]:
def add_region_column(df):
    state_groups = {
        'Northern Region': ['Jammu and Kashmir', 'Himachal Pradesh', 'Punjab', 'Chandigarh', 'Uttarakhand', 'Ladakh', 'Delhi', 'Haryana'],
        'Central Region': ['Uttar Pradesh', 'Madhya Pradesh', 'Chhattisgarh'],
        'Western Region': ['Rajasthan', 'Gujarat', 'Dadra and Nagar Haveli and Daman and Diu', 'Maharashtra'],
        'Eastern Region': ['Bihar', 'Jharkhand', 'Odisha', 'West Bengal', 'Sikkim'],
        'Southern Region': ['Andhra Pradesh', 'Telangana', 'Karnataka', 'Kerala', 'Tamil Nadu', 'Puducherry', 'Goa', 'Lakshadweep', 'Andaman and Nicobar Islands'],
        'North-Eastern Region': ['Assam', 'Meghalaya', 'Manipur', 'Nagaland', 'Tripura', 'Arunachal Pradesh', 'Mizoram']
    }
    
    df['Region'] = df['State'].map({state: region for region, states in state_groups.items() for state in states})
    return df

In [18]:
for df_name in df_list:
    df = globals()[df_name]
    add_region_column(df)

# Columnwise null-count and duplicated_rows-count

In [19]:
for df_name in df_list:
    df = globals()[df_name]
    print(f"{df_name}:")
    print(f"Null count: \n{df.isnull().sum()}")
    print(f"Duplicated rows count: \n{df.duplicated().sum()}")
    print(df.shape)
    print("\n", 25 * "_", "\n")

agg_trans_df:
Null count: 
State                 0
Year                  0
Quarter               0
Transaction_type      0
Transaction_count     0
Transaction_amount    0
Region                0
dtype: int64
Duplicated rows count: 
0
(3594, 7)

 _________________________ 

agg_user_df:
Null count: 
State                0
Year                 0
Quarter              0
Brand                0
Transaction_count    0
Percentage           0
Region               0
dtype: int64
Duplicated rows count: 
0
(6732, 7)

 _________________________ 

map_trans_df:
Null count: 
State                 0
Year                  0
Quarter               0
District              0
Transaction_count     0
Transaction_amount    0
Latitude              0
Longitude             0
Region                0
dtype: int64
Duplicated rows count: 
0
(14636, 9)

 _________________________ 

map_user_df:
Null count: 
State               0
Year                0
Quarter             0
District            0
Registered_users    0
A

# Understanding the dataframes

In [20]:
print('DATAFRAME INFO:\n')

for df_name in df_list:
    df = globals()[df_name]
    print(df_name + ':\n')
    df.info()
    print("\n", 45 * "_", "\n")

DATAFRAME INFO:

agg_trans_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594 entries, 0 to 3593
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               3594 non-null   object 
 1   Year                3594 non-null   object 
 2   Quarter             3594 non-null   int64  
 3   Transaction_type    3594 non-null   object 
 4   Transaction_count   3594 non-null   int64  
 5   Transaction_amount  3594 non-null   float64
 6   Region              3594 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 196.7+ KB

 _____________________________________________ 

agg_user_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              6732 non-null   object 
 1   Year               6732 non-null   obj

# Dropping rows with null values

In [21]:
# 'top_trans_pin_df' seems to have two null values and they are not of significant proportion so dropping them;

top_trans_pin_df.dropna(axis = 'index', inplace = True)
top_trans_pin_df.isnull().sum()

State                 0
Year                  0
Quarter               0
Pincode               0
Transaction_count     0
Transaction_amount    0
Region                0
dtype: int64

# Changing datatype across all dataframes

In [22]:
# Year column in all the dataframes seems to be of object dtype so changing it to int object so as to push into MySQL as year;

for df_name in df_list:
    df = globals()[df_name]
    df['Year'] = df['Year'].astype('int')

# Outlier count across all dataframes

In [23]:
# Everything seems to be alright as far as dtypes and nullvalues are concerned so checking for outliers
# Function to check for outliers

def count_outliers(df):
    outliers = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        if col in ['Transaction_count', 'Transaction_amount']:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5 * iqr)
            lower_bound = q1 - (1.5 * iqr)
            outliers[col] = len(df[(df[col] > upper_bound) | (df[col] < lower_bound)])
        else:
            continue
    return outliers

In [24]:
print('OUTLIER COUNT ACROSS DATAFRAMES:\n')

for df_name in df_list:
    df = globals()[df_name]
    outliers = count_outliers(df)
    if len(outliers) == 0:
        pass
    else:
        print(df_name, ":\n\n", outliers, "\n")
        print("\n", 55 * "_", "\n")

OUTLIER COUNT ACROSS DATAFRAMES:

agg_trans_df :

 {'Transaction_count': 652, 'Transaction_amount': 660} 


 _______________________________________________________ 

agg_user_df :

 {'Transaction_count': 893} 


 _______________________________________________________ 

map_trans_df :

 {'Transaction_count': 1811, 'Transaction_amount': 1771} 


 _______________________________________________________ 

top_trans_dist_df :

 {'Transaction_count': 734, 'Transaction_amount': 743} 


 _______________________________________________________ 

top_trans_pin_df :

 {'Transaction_count': 999, 'Transaction_amount': 995} 


 _______________________________________________________ 



# Unique value count across all dataframes

In [25]:
# Function to check for unique value counts and print if count less than 10;

def unique_value_count(df, exclude_cols=[]):
    for col in df.columns:
        if col in exclude_cols:
            continue
        unique_vals = df[col].nunique()
        print(f"{col}: {unique_vals} unique values")
        if unique_vals < 10:
            print(df[col].unique())

In [26]:
print('UNIQUE VALUE COUNT ACROSS DATAFRAMES; \n')

for df_name in df_list:
    df = globals()[df_name]
    print(df_name, ":\n")
    unique_value_count(df, exclude_cols = ['State', 'Year', 'Quarter', 'Percentage'])
    print("\n", 55 * "_", "\n")

UNIQUE VALUE COUNT ACROSS DATAFRAMES; 

agg_trans_df :

Transaction_type: 5 unique values
['Recharge & bill payments' 'Peer-to-peer payments' 'Merchant payments'
 'Financial Services' 'Others']
Transaction_count: 3548 unique values
Transaction_amount: 3594 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region']

 _______________________________________________________ 

agg_user_df :

Brand: 20 unique values
Transaction_count: 6501 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region']

 _______________________________________________________ 

map_trans_df :

District: 727 unique values
Transaction_count: 14566 unique values
Transaction_amount: 14636 unique values
Latitude: 533 unique values
Longitude: 540 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Reg

# Creating CSV files out of the refined dataframes

In [27]:
def save_dfs_as_csv(df_list):
    subfolder = 'ExtData'
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)
        
    for df_name in df_list:
        df = globals()[df_name]
        file_path = os.path.join(subfolder, df_name.replace('_df', '') + '.csv')
        df.to_csv(file_path, index=False)

# Calling function to execute

save_dfs_as_csv(df_list)

# SQL part

# Establishing connection and creating cursor

In [28]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [29]:
conn = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = "rp#$9882"
)

cursor = conn.cursor()

# Database creation

In [30]:
cursor.execute("DROP DATABASE IF EXISTS phonepe_pulse")

cursor.execute("CREATE DATABASE phonepe_pulse")

cursor.execute("USE phonepe_pulse")

# Creating tables

In [31]:
cursor.execute('''CREATE TABLE agg_trans (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    Transaction_type VARCHAR(255),
                    Transaction_count INTEGER,
                    Transaction_amount FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, Transaction_type(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE agg_user (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    Brand VARCHAR(255),
                    Transaction_count INTEGER,
                    Percentage FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, Brand(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE map_trans (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    District VARCHAR(255),
                    Transaction_count INTEGER,
                    Transaction_amount FLOAT,
                    Latitude FLOAT,
                    Longitude FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, District(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE map_user (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    District VARCHAR(255),
                    Registered_users INTEGER,
                    App_opens INTEGER,
                    Latitude FLOAT,
                    Longitude FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, District(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE top_trans_dist (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    District VARCHAR(255),
                    Transaction_count INTEGER,
                    Transaction_amount FLOAT,
                    Latitude FLOAT,
                    Longitude FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, District(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE top_trans_pin (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    Pincode VARCHAR(255),
                    Transaction_count INTEGER,
                    Transaction_amount FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, Pincode(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE top_user_dist (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    District VARCHAR(255),
                    Registered_users INTEGER,
                    Latitude FLOAT,
                    Longitude FLOAT,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, District(255), Region(255))
                 )''')

cursor.execute('''CREATE TABLE top_user_pin (
                    State VARCHAR(255),
                    Year YEAR,
                    Quarter INTEGER,
                    Pincode VARCHAR(255),
                    Registered_users INTEGER,
                    Region VARCHAR(255),
                    PRIMARY KEY (State(255), Year, Quarter, Pincode(255), Region(255))
                 )''')

# Pushing data into MySQL

In [32]:
def push_data_into_mysql(conn, cursor, dfs, table_columns):
    for table_name in dfs.keys():
        df = dfs[table_name]
        columns = table_columns[table_name]
        placeholders = ', '.join(['%s'] * len(columns))
        query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
        for _, row in df.iterrows():
            data = tuple(row[column] for column in columns)
            cursor.execute(query, data)
        conn.commit()
    print("Data successfully pushed into MySQL tables")

In [33]:
# Mapping my_sql tables to pandas dataframes that we have created earlier

dfs = {
    'agg_trans': agg_trans_df,
    'agg_user': agg_user_df,
    'map_trans': map_trans_df,
    'map_user': map_user_df,
    'top_trans_dist': top_trans_dist_df,
    'top_trans_pin': top_trans_pin_df,
    'top_user_dist': top_user_dist_df,
    'top_user_pin': top_user_pin_df
}

# Mapping table name to associated columns for each table

table_columns = {
    'agg_trans': list(agg_trans_df.columns),
    'agg_user': list(agg_user_df.columns),
    'map_trans': list(map_trans_df.columns),
    'map_user': list(map_user_df.columns),
    'top_trans_dist': list(top_trans_dist_df.columns),
    'top_trans_pin': list(top_trans_pin_df.columns),
    'top_user_dist': list(top_user_dist_df.columns),
    'top_user_pin': list(top_user_pin_df.columns)
}

In [34]:
push_data_into_mysql(conn, cursor, dfs, table_columns)

Data successfully pushed into MySQL tables


# Checking whether shape of tables and dataframes are equal or not

In [35]:
# Get list of tables in database

cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Loop through tables and get count of rows and columns in MySQL

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    cursor.execute(f"SELECT COUNT(*) FROM information_schema.columns WHERE table_name='{table_name}'")
    column_count = cursor.fetchone()[0]

    # Check if shape of DataFrame matches count of rows and columns in table

    df = dfs[table_name]
    if df.shape == (row_count, column_count):
        print(f"{table_name} table has {row_count} rows and {column_count} columns and shape matches DataFrame.")
    else:
        print(f"{table_name} table has {row_count} rows and {column_count} columns but shape does not match DataFrame.")

cursor.close()
conn.close()

agg_trans table has 3594 rows and 7 columns and shape matches DataFrame.
agg_user table has 6732 rows and 7 columns and shape matches DataFrame.
map_trans table has 14636 rows and 9 columns and shape matches DataFrame.
map_user table has 14640 rows and 9 columns and shape matches DataFrame.
top_trans_dist table has 5920 rows and 9 columns and shape matches DataFrame.
top_trans_pin table has 7137 rows and 7 columns and shape matches DataFrame.
top_user_dist table has 5920 rows and 8 columns and shape matches DataFrame.
top_user_pin table has 7140 rows and 6 columns and shape matches DataFrame.
